In [10]:
!python -m spacy download en_core_web_sm

[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


2022-10-29 15:27:36.850694: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-10-29 15:27:36.850789: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-29 15:27:41.907516: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'nvcuda.dll'; dlerror: nvcuda.dll not found
2022-10-29 15:27:41.907554: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-10-29 15:27:41.913394: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: Fuzel
2022-10-29 15:27:41.913626: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: Fuzel


In [8]:
!pip install negspacy

  Created wheel for negspacy: filename=negspacy-1.0.3-py3-none-any.whl size=10434 sha256=3ffa193a3fb3b5fd88f4b5007242c591ac3a4296643622d27bb9374fb7b0915e
  Stored in directory: c:\users\fuzel\appdata\local\pip\cache\wheels\ec\4b\5b\ae823d4b232c1342724ec93edab9f3dca6403679d93e0daa25
Successfully built negspacy


In [6]:
!pip install spacy

  Attempting uninstall: smart-open
    Found existing installation: smart-open 5.1.0
    Uninstalling smart-open-5.1.0:
      Successfully uninstalled smart-open-5.1.0


In [2]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import collections
import re
from nltk.stem.porter import PorterStemmer
from sklearn.metrics import accuracy_score, classification_report
import spacy
from negspacy.negation import Negex

In [3]:
stop_words = set(stopwords.words('english'))

In [4]:
l = ["haven't","hadn't","not","don't","aren't","less","none","couldn't","doesn't","hasn't","isn't","mightn't","mustn't","needn't","shan't","shouldn't","wasn't","weren't","won't","wouldn't",""]
for w in l:
    try:
        stop_words.remove(w)
    except:
        pass

In [5]:
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("negex", config={"ent_types":["PERSON","ORG"]})

# We are using corona, economic, FNC challenge datasets to run through the model and get the accuracy

In [21]:
#Below the dataframe is reading the corona dataset

#df1 = pd.read_csv('Corona_Bodies.csv',encoding='latin-1')
#df2 = pd.read_csv('Corona_stance.csv',encoding='latin-1')
#combined_df = df2
#combined_df['articleBody'] = df1['articleBody']

#Below the dataframe is reading the economic dataset

#df1 = pd.read_csv('Economic_Bodies.csv',encoding='latin-1')
#df2 = pd.read_csv('Economic_Stance.csv',encoding='latin-1')
#combined_df = df2
#combined_df['articleBody'] = df1['articleBody']
#combined_df

#Below the dataframe is reading the FNC challenge test dataset

df1 = pd.read_csv('test_bodies.csv')
df2 = pd.read_csv('test_stances_unlabeled.csv')
combined_df = df1.merge(df2,on='Body ID')

In [22]:
a = []
b = []
c = []
count = 0
ps = PorterStemmer()
for _id in range(0,combined_df.shape[0]):
    head_line = combined_df.iloc[_id]['Headline']
    art_body = combined_df.iloc[_id]['articleBody']
    head_line = re.sub(r"\n|\'|-|,|\)|\(|\:|\?|@|‘|’|\*|\"|\$",'',head_line)
    head_line = ' '.join([i for i in head_line.split(' ') if i not in stop_words])
    head_line = ' '.join([ps.stem(i) for i in head_line.split(' ')])
    art_body = re.sub(r"\n|\'|-|,|\)|\(|\:|\?|@|‘|’|\*|\"|\$",'',art_body)
    art_body = ' '.join([i.lower() for i in art_body.split(' ') if i not in stop_words])
    art_body = ' '.join([ps.stem(i) for i in art_body.split(' ')])
    article_counter = collections.Counter(art_body)
    neg_body = []
    doc1 = nlp(art_body)
    for e in doc1.ents:
        neg_body.append(tuple([e.text,str(e._.negex)]))
    neg_head_line = []
    doc = nlp(head_line)
    for e in doc.ents:
        neg_head_line.append(tuple([e.text,str(e._.negex)]))
    count = 0
    head_line_counter = collections.Counter(head_line)
    try:
        for k in head_line_counter.keys():
            count = count+article_counter[k]
    except:
        count = count + 0
    if count<=1:
        a.append(head_line)
        b.append(art_body)
        c.append('unrelated')
    else:
        if count>=len(head_line) and count>=5:
            a.append(head_line)
            b.append(art_body)
            c.append('agree')
        elif count<len(head_line) and count >= 2:
            a.append(head_line)
            b.append(art_body)
            c.append('discuss')
        else:
            for index, tup in enumerate(neg_head_line):
                for i,j in enumerate(neg_body):
                    if index == i:
                        if tup != j:
                            a.append(head_line)
                            b.append(art_body)
                            c.append('disagree')

In [23]:
df = pd.DataFrame()
df['Headline'] = a
df['articleBody'] = b
df['Stance'] = c

In [24]:
df['Stance'].unique()

,Headline,articleBody,Stance
0,appl instal safe instor protect gold watch edit,alsisi deni isra report state offer extend gaz...,agree
1,elsisi deni claim hell give sinai land palesti...,alsisi deni isra report state offer extend gaz...,discuss
2,appl keep gold watch edit special instor safe,alsisi deni isra report state offer extend gaz...,agree
3,appl store keep gold “edition” appl watch cust...,alsisi deni isra report state offer extend gaz...,discuss
4,south korean woman hair eaten robot vacuum cle...,alsisi deni isra report state offer extend gaz...,discuss
...,...,...,...
25408,a sign that obamacar exchang are fail,rememb much republican want repeal obamacareth...,agree
25409,republican call obamacar failure. these 7 char...,rememb much republican want repeal obamacareth...,agree
25410,cbo altern fact show obamacar unsustain,rememb much republican want repeal obamacareth...,agree
25411,whi obamacar fail,rememb much republican want repeal obamacareth...,agree


# Below code cells are to check accuracy of corona and economic datasets. To check accuracy of FNC dataset scroll down a bit

In [28]:
accuracy_score(combined_df['Stance'],df['Stance'])

0.38

In [29]:
print(classification_report(combined_df['Stance'],df['Stance']))

              precision    recall  f1-score   support

       agree       0.38      1.00      0.55        76
    disagree       0.00      0.00      0.00        40
     discuss       0.00      0.00      0.00        39
   unrelated       0.00      0.00      0.00        45

    accuracy                           0.38       200
   macro avg       0.10      0.25      0.14       200
weighted avg       0.15      0.38      0.21       200



C:\Users\fuzel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\fuzel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\fuzel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Run below code cells only for comparing FNC dataset results from the model

In [32]:
fnc_test = pd.read_csv('fnc_testdata_predictions.csv')
fnc_test

,Headline,Body ID,stance
0,Ferguson riots: Pregnant woman loses eye after...,2008,unrelated
1,Crazy Conservatives Are Sure a Gitmo Detainee ...,1550,unrelated
2,A Russian Guy Says His Justin Bieber Ringtone ...,2,unrelated
3,"Zombie Cat: Buried Kitty Believed Dead, Meows ...",1793,unrelated
4,Argentina's President Adopts Boy to End Werewo...,37,unrelated
...,...,...,...
25408,The success of the Affordable Care Act is a hu...,2582,agree
25409,The success of the Affordable Care Act is a hu...,2583,discuss
25410,The success of the Affordable Care Act is a hu...,2584,unrelated
25411,The success of the Affordable Care Act is a hu...,2585,discuss


In [33]:
accuracy_score(fnc_test['stance'],df['Stance'])

0.27619722189430607

In [29]:
print(classification_report(fnc_test['stance'],df['Stance']))

C:\Users\fuzel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

       agree       0.06      0.97      0.11      1495
    disagree       0.00      0.00      0.00        36
     discuss       0.20      0.03      0.05      5470
   unrelated       0.00      0.00      0.00     18412

    accuracy                           0.06     25413
   macro avg       0.06      0.25      0.04     25413
weighted avg       0.05      0.06      0.02     25413



C:\Users\fuzel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\fuzel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


End of the code